# Advanced Read Operations


ParquetDB offers a `read` method that enables a range of advanced data retrieval options. These options include:
- Filtering data by column values (predicate pushdown),
- Selecting or excluding specific columns,
- Batch processing for optimized memory usage,
- Rebuilding nested structures for hierarchical data, and more.

In this notebook, we will explore how to leverage **advanced read** capabilities to efficiently retrieve data that meets your exact requirements.



The `read` method has the following signature:

```python
def read(
    self,
    ids: List[int] = None,
    columns: List[str] = None,
    filters: List[pc.Expression] = None,
    load_format: str = "table",
    batch_size: int = None,
    include_cols: bool = True,
    rebuild_nested_struct: bool = False,
    rebuild_nested_from_scratch: bool = False,
    load_config: LoadConfig = LoadConfig(),
    normalize_config: NormalizeConfig = NormalizeConfig(),
) -> Union[pa.Table, Generator, Any]:
    ...
```

In [1]:
import pprint
import time
import os
from pathlib import Path
import shutil
import pyarrow as pa
from parquetdb import ParquetDB, LoadConfig, NormalizeConfig
from parquetdb.utils.general_utils import generate_similar_data

# Define a simple template data entry
template_dict = {
    "float_field": 10,
    "int_field": 10,
    "name": "item",
    "nested_value": {"value": 10, "name": "item"},
    "list_field": [1, 2, 3],
}
for x in range(500):
    template_dict[f"column_{x}"] = "test"

template = [template_dict]

# Generate multiple data entries
num_entries = 100000  # Feel free to adjust this
data = generate_similar_data(template, num_entries)

print("Generated Data:")
pprint.pprint(data[0])

ROOT_DIR = Path(".")
DATA_DIR = ROOT_DIR / "data"

if DATA_DIR.exists():
    shutil.rmtree(DATA_DIR)
    
db_path = ROOT_DIR / "ParquetDB"

db = ParquetDB(db_path=db_path)

db.create(data)
print(db)
data = None

Generated Data:
{'column_0': 'test_64',
 'column_1': 'test_83',
 'column_10': 'test_71',
 'column_100': 'test_68',
 'column_101': 'test_65',
 'column_102': 'test_67',
 'column_103': 'test_60',
 'column_104': 'test_73',
 'column_105': 'test_10',
 'column_106': 'test_74',
 'column_107': 'test_73',
 'column_108': 'test_60',
 'column_109': 'test_58',
 'column_11': 'test_39',
 'column_110': 'test_100',
 'column_111': 'test_18',
 'column_112': 'test_42',
 'column_113': 'test_62',
 'column_114': 'test_58',
 'column_115': 'test_16',
 'column_116': 'test_84',
 'column_117': 'test_32',
 'column_118': 'test_11',
 'column_119': 'test_28',
 'column_12': 'test_73',
 'column_120': 'test_49',
 'column_121': 'test_12',
 'column_122': 'test_22',
 'column_123': 'test_52',
 'column_124': 'test_37',
 'column_125': 'test_1',
 'column_126': 'test_47',
 'column_127': 'test_37',
 'column_128': 'test_12',
 'column_129': 'test_15',
 'column_13': 'test_36',
 'column_130': 'test_35',
 'column_131': 'test_54',
 'co

## Basic Read

A straightforward read operation without any parameters returns **all** data in a `pyarrow.Table` format (default).


In [2]:
start_time = time.time()
data = db.read()
print(data.shape)
end_time = time.time()
print(f"Total memory allocated: {pa.total_allocated_bytes()/1024/1024} MB")
print(f"Time taken to read all data: {end_time - start_time} seconds")
data = None

(100000, 507)
Total memory allocated: 572.2216796875 MB
Time taken to read all data: 0.3734889030456543 seconds


## Column Selection

### Read Specific Columns

To retrieve specific columns, you can pass a list of column names to the `columns` parameter.

In [3]:
start_time = time.time()
data = db.read(columns=["int_field"])
print(data.shape)
end_time = time.time()
print(f"Total memory allocated: {pa.total_allocated_bytes()/1024/1024} MB")
print(f"Time taken to read all data: {end_time - start_time} seconds")
data = None

(100000, 1)
Total memory allocated: 0.77490234375 MB
Time taken to read all data: 0.05100083351135254 seconds


### Read Nested Columns

As you can see, using column selection it is much faster and less memory intensive as only the specified columns are loaded into memory.

ParquetDB by defualts flattens nested structures. If you want the read fields from the nested structure you can specify it by using the syntax `nested_value.field_name`.


In [4]:
start_time = time.time()
data = db.read(columns=["nested_value.name"])
print(data.shape)
end_time = time.time()
print(f"Total memory allocated: {pa.total_allocated_bytes()/1024/1024} MB")
print(f"Time taken to read all data: {end_time - start_time} seconds")
data = None

(100000, 1)
Total memory allocated: 1.12738037109375 MB
Time taken to read all data: 0.046912431716918945 seconds


### Do not read selected columns

If you want to read all columns but not include the selected columns you can set the `include_cols` parameter to `False`.


In [5]:
start_time = time.time()
data = db.read(columns=["int_field"], include_cols=False)
print(data.shape)
end_time = time.time()
column_names = data.column_names
assert "int_field" not in column_names
print(f"Total memory allocated: {pa.total_allocated_bytes()/1024/1024} MB")
print(f"Time taken to read all data: {end_time - start_time} seconds")
data = None

(100000, 506)
Total memory allocated: 571.6526489257812 MB
Time taken to read all data: 0.35604023933410645 seconds


## Filtering Data

You can filter data by using the `filters` parameter. This parameter accepts a list of pyarrow expressions. ParquetDB will combine the filters using the `and` operator.

> For more information on how to create pyarrow expressions, please refer to the [pyarrow documentation](https://arrow.apache.org/docs/python/compute.html#filtering-by-expressions).

In [6]:
import pyarrow.compute as pc

start_time = time.time()
data = db.read(filters=[pc.field("int_field") < 10])
print(data.shape)
end_time = time.time()
print(f"Total memory allocated: {pa.total_allocated_bytes()/1024/1024} MB")
print(f"Time taken to read all data: {end_time - start_time} seconds")
data = None

(47684, 507)
Total memory allocated: 425.08599853515625 MB
Time taken to read all data: 0.4452495574951172 seconds


Again, since we do not read all the rows, the memory usage is much lower and the time taken is also faster.

You can can still choose which columns to read while filtering.

In [7]:
start_time = time.time()
data = db.read(columns=["nested_value.name"], filters=[pc.field("int_field") < 10])
print(data.shape)
end_time = time.time()
print(f"Total memory allocated: {pa.total_allocated_bytes()/1024/1024} MB")
print(f"Time taken to read all data: {end_time - start_time} seconds")
data = None

(47684, 1)
Total memory allocated: 1.09210205078125 MB
Time taken to read all data: 0.03300309181213379 seconds


## Rebuilding Nested Structures

ParquetDB by default flattens nested structures. If you want to read the nested structure to read the nested data in its original form, you can set the `rebuild_nested_struct` parameter to `True`. This will create a new files that will contain the nested data. 

> Note: direct updates, creates, or deletes to the nested data will not be reflected in the original files. You will have to rebuild the nested structure from scratch. by using the `rebuild_nested_from_scratch` parameter.


In [10]:
start_time = time.time()
data = db.read(rebuild_nested_struct=True)
print(data.shape)
column_names = data.column_names
assert "nested_value.name" not in column_names
assert "nested_value.value" not in column_names
assert "nested_value" in column_names
end_time = time.time()
print(f"Total memory allocated: {pa.total_allocated_bytes()/1024/1024} MB")
print(f"Time taken to read all data: {end_time - start_time} seconds")
data = None

(100000, 506)
Total memory allocated: 528.3989868164062 MB
Time taken to read all data: 0.29286813735961914 seconds


Since the data is in nested format, to read specific nested fields slightly change. Now you can directly call the parent field and it will return all children fields. 

In [13]:
start_time = time.time()
data = db.read(columns=["nested_value"], rebuild_nested_struct=True).to_pandas()
print(data.shape)
column_names = data.columns
print(data.head())
assert "nested_value" in column_names
end_time = time.time()
print(f"Time taken to read all data: {end_time - start_time} seconds")
data = None

(100000, 1)
                       nested_value
0  {'name': 'item_13', 'value': 16}
1  {'name': 'item_83', 'value': 13}
2  {'name': 'item_27', 'value': 14}
3  {'name': 'item_20', 'value': 20}
4  {'name': 'item_87', 'value': 12}
Time taken to read all data: 0.06846165657043457 seconds


You can still specify the child field in the previous form

In [14]:
start_time = time.time()
data = db.read(columns=["nested_value.name"], rebuild_nested_struct=True).to_pandas()
print(data.shape)
column_names = data.columns
print(data.head())
end_time = time.time()
print(f"Time taken to read all data: {end_time - start_time} seconds")
data = None

(100000, 1)
      name
0  item_13
1  item_83
2  item_27
3  item_20
4  item_87
Time taken to read all data: 0.03645682334899902 seconds


When the data is in nested format, creating pyarrow expressions for filtering is a bit different


In [15]:
start_time = time.time()
data = db.read(
    columns=["nested_value.name"],
    filters=[pc.field("nested_value", "name") == "item_20"],
    rebuild_nested_struct=True,
).to_pandas()
print(data.shape)
column_names = data.columns
print(data.head())
end_time = time.time()
print(f"Time taken to read all data: {end_time - start_time} seconds")
data = None

(1009, 1)
      name
0  item_20
1  item_20
2  item_20
3  item_20
4  item_20
Time taken to read all data: 0.04416155815124512 seconds


## Batch Processing

ParquetDB also supports batch processing. This is useful when you want to process data in smaller chunks to avoid memory issues. This can be done by setting the `load_format="batches"` and setting the `batch_size` parameter.

When this is used, the `read` method will return a generator that yields `pyarrow.RecordBatch` objects, which are similar to `pyarrow.Table` objects.


In [19]:
start_time = time.time()
generator = db.read(load_format="batches", batch_size=10000)
print(generator)
end_time = time.time()
print(f"Total memory allocated: {pa.total_allocated_bytes()/1024/1024} MB")
print(f"Time taken to read all data: {end_time - start_time} seconds")

Total memory allocated: 392.501708984375 MB
Time taken to read all data: 0.019051790237426758 seconds


Now, you can iterate over the generator to get the data in batches.

In [20]:
for record_batch in generator:
    print(record_batch.shape)

(10000, 507)
(10000, 507)
(10000, 507)
(2768, 507)
(10000, 507)
(10000, 507)
(10000, 507)
(2768, 507)
(10000, 507)
(10000, 507)
(10000, 507)
(2768, 507)
(1696, 507)
(10000, 507)
(10000, 507)
(10000, 507)
(2768, 507)
(10000, 507)
(10000, 507)
(10000, 507)
(2768, 507)
(10000, 507)
(10000, 507)
(10000, 507)
(2768, 507)
(1696, 507)


More configurations can be used to control the batching process by adding the `load_config` parameter which is an instance of `LoadConfig`.

```python
@dataclass
class LoadConfig:
    """
    Configuration for loading data, specifying columns, filters, batch size, and memory usage.

    Parameters
    ----------
    batch_size : int, optional
        The number of rows to process in each batch (default: 131_072).
    batch_readahead : int, optional
        The number of batches to read ahead in a file (default: 16).
    fragment_readahead : int, optional
        The number of files to read ahead, improving IO utilization at the cost of RAM usage (default: 4).
    fragment_scan_options : Optional[pa.dataset.FragmentScanOptions], optional
        Options specific to a particular scan and fragment type, potentially changing across scans.
    use_threads : bool, optional
        Whether to use maximum parallelism determined by available CPU cores (default: True).
    memory_pool : Optional[pa.MemoryPool], optional
        The memory pool for allocations. Defaults to the system's default memory pool.
    """

    batch_size: int = 131_072
    batch_readahead: int = 16
    fragment_readahead: int = 4
    fragment_scan_options: Optional[pa.dataset.FragmentScanOptions] = None
    use_threads: bool = True
    memory_pool: Optional[pa.MemoryPool] = None
```


In [27]:
load_config = LoadConfig(batch_size=10000, batch_readahead=1)
generator = db.read(load_format="batches", load_config=load_config)
start_time = time.time()
for record_batch in generator:
    batch = record_batch.shape
end_time = time.time()
print(
    f"Time taken to read all data when batch_readahead is 1: {end_time - start_time} seconds"
)

load_config = LoadConfig(batch_size=10000, batch_readahead=16)
generator = db.read(load_format="batches", load_config=load_config)
start_time = time.time()
for record_batch in generator:
    batch = record_batch.shape
end_time = time.time()
print(
    f"Time taken to read all data when batch_readahead is 4: {end_time - start_time} seconds"
)

Time taken to read all data when batch_readahead is 1: 0.6078670024871826 seconds
Time taken to read all data when batch_readahead is 4: 0.5378696918487549 seconds


This difference is small because this dataset is small, but it can be more noticeable for larger datasets.